In [12]:
import pandas as pd
from pathlib import Path
from scripts.normalizzatori import normalizza_descrizione,normalizza_titolo,normalizza_valutazioni

In [13]:
base_dir = Path().resolve().parent  # root
data_path = base_dir / 'data'
data_clean_path = base_dir / 'data_clean'
data={}

In [14]:
for file in data_path.glob('*.csv'):
    nome = file.stem
    data[nome] = pd.read_csv(file)

droppiamo tutti i film che non hanno nome 

In [15]:
data['the_oscar_awards'] = data['the_oscar_awards'].dropna(subset='film')
data['rotten_tomatoes_reviews'] = data['rotten_tomatoes_reviews'].dropna(subset='movie_title')

normalizzazione descrizioni e recensioni di rotten_tomato 

In [16]:
data['rotten_tomatoes_reviews'].review_score = data['rotten_tomatoes_reviews'].review_score.apply(normalizza_valutazioni)
data['movies']['description'] = normalizza_descrizione(data['movies']['description'])
data['movies'].date =  data['movies'].date.astype('Int64')

matching ID tra rotten_tomatoes_reviews, the_oscar_awards e movies.id

In [17]:
data['movies']['name_clean'] = data['movies']['name'].apply(normalizza_titolo)
data['rotten_tomatoes_reviews']['movie_name_clean'] = data['rotten_tomatoes_reviews']['movie_title'].apply(lambda x: normalizza_titolo(x, parentesi=True))
data['the_oscar_awards']['film_clean'] = data['the_oscar_awards']['film'].apply(lambda x: normalizza_titolo(x, parentesi=True))

In [18]:
movies_unique = data['movies'].drop_duplicates(subset='name_clean')

In [19]:
data['rotten_tomatoes_reviews'] = data['rotten_tomatoes_reviews'].merge(movies_unique[['id', 'name_clean']], left_on='movie_name_clean', right_on='name_clean', how='left').drop(columns=['movie_name_clean','name_clean'])
data['the_oscar_awards'] = data['the_oscar_awards'].merge(movies_unique[['id', 'name_clean']], left_on='film_clean', right_on='name_clean', how='left').drop(columns=['film_clean','name_clean'])
data['movies'] = data['movies'].drop(columns='name_clean')

In [20]:
data['the_oscar_awards'].id = data['the_oscar_awards'].id.astype('Int64')
data['rotten_tomatoes_reviews'].id = data['rotten_tomatoes_reviews'].id.astype('Int64')

In [21]:
for name,df in data.items():
    if name != 'movie':
        df= df.rename(columns={'id':'movie_id'})

    df.to_csv(data_clean_path / f'{name}.csv', index=False)

FILM PERSI 

3.2929974211465978 oscar persi  
4.87719298245614 rotten persi 